In [6]:
from wallstreet import wallstreet

In [7]:
stock = wallstreet.Stock('AAPL')
df = stock.historical(days_back=5)
print(df)

        Date        Open        High         Low       Close   Adj Close  \
0 2022-09-08  154.639999  156.360001  152.679993  154.460007  154.460007   
1 2022-09-09  155.470001  157.820007  154.750000  157.369995  157.369995   
2 2022-09-12  159.589996  164.259995  159.300003  163.429993  163.429993   
3 2022-09-13  159.899994  160.539993  153.380997  153.839996  153.839996   

      Volume  
0   84923800  
1   68028800  
2  104956000  
3  122159350  


In [12]:
from sqlalchemy import MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from setup_psql_enironment import get_database
import pandas as pd

In [3]:
def create_database():
    # Setup environment and create a session
    db = get_database()
    Session = sessionmaker(bind=db)
    meta = MetaData(bind=db)
    session = Session()

In [4]:
db = get_database()
Session = sessionmaker(bind=db)
meta = MetaData(bind=db)
session = Session()

Connected to PostgreSQL database!


In [8]:
with db.connect() as connection:
    with connection.begin():
        tickers_with_id = connection.execute("SELECT ticker, id FROM security").fetchall()
        for ticker, _id in tickers_with_id:
            stock = wallstreet.Stock(ticker)
            df = stock.historical(days_back=5)
            print(df)
            for index, row in df.iterrows():
                pass
                # print(row['Open'])
                # connection.execute("INSERT INTO daily_price (data_vendor_id, ticker_id, price_date, open_price, high_price, low_price, close_price, adj_close_price, volume) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)")

print(tickers_with_id[:5])

2022-09-14 01:09:38,920 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-14 01:09:38,922 INFO sqlalchemy.engine.Engine SELECT ticker, id FROM security
2022-09-14 01:09:38,923 INFO sqlalchemy.engine.Engine [raw sql] {}
        Date        Open        High         Low       Close   Adj Close  \
0 2022-09-08  130.929993  135.270004  130.710007  135.190002  135.190002   
1 2022-09-09  135.979996  137.919998  135.429993  137.630005  137.630005   
2 2022-09-12  137.529999  140.000000  137.229996  139.869995  139.869995   
3 2022-09-13  136.320007  137.410004  133.440002  133.539993  133.539993   

    Volume  
0  1355100  
1  2425200  
2  1851100  
3  1553752  
        Date       Open       High        Low      Close  Adj Close   Volume
0 2022-09-08  47.980000  48.939999  47.150002  48.860001  48.860001  5264300
1 2022-09-09  50.400002  53.070000  50.270000  52.619999  52.619999  7268800
2 2022-09-12  53.400002  53.790001  51.560001  52.340000  52.340000  5283200
3 2022-09-13  50.34000

KeyboardInterrupt: 

In [16]:
with db.connect() as connection:
    with connection.begin():
        df_orders      = pd.read_sql("SELECT sector, id FROM security ", connection)

        
print("HE", df_orders)


2022-09-14 12:08:57,072 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-14 12:08:57,127 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-09-14 12:08:57,131 INFO sqlalchemy.engine.Engine [cached since 6778s ago] {'name': 'SELECT sector, id FROM security '}
2022-09-14 12:08:57,293 INFO sqlalchemy.engine.Engine SELECT sector, id FROM security 
2022-09-14 12:08:57,294 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-09-14 12:08:57,491 INFO sqlalchemy.engine.Engine COMMIT
HE                       sector    id
0                Industrials   335
1                Industrials   336
2                    Finance   337
3     Consumer Discretionary   338
4                Industrials   339
...                      ...   ...
2311                     nan  2646
2312             Health Care  2647
2313                 Finance  2648
2314             Industrials  2649
2315    

In [24]:
df_orders.groupby('sector').mean().reset_index()

,sector,id
0,Basic Materials,1666.769231
1,Consumer Discretionary,1513.484568
2,Consumer Staples,1435.325000
3,Energy,1736.938462
4,Finance,1508.317073
5,Health Care,1397.342640
6,Industrials,1467.198225
7,Miscellaneous,1662.857143
8,Real Estate,1556.714286
9,Technology,1533.965649


In [26]:
dff = df_orders[df_orders['sector']=='Energy']
dff

,sector,id
23,Energy,358
154,Energy,488
388,Energy,720
389,Energy,721
404,Energy,736
...,...,...
2273,Energy,2608
2279,Energy,2614
2289,Energy,2624
2304,Energy,2639


In [32]:
with db.connect() as connection:
    with connection.begin():
        df_orders2      = pd.read_sql("SELECT date, open FROM price WHERE ticker_id in (SELECT id FROM security WHERE ticker='A')", connection)
df_orders2


2022-09-14 13:35:46,097 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-14 13:35:46,129 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-09-14 13:35:46,134 INFO sqlalchemy.engine.Engine [cached since 1.199e+04s ago] {'name': "SELECT date, open FROM price WHERE ticker_id in (SELECT id FROM security WHERE ticker='A')"}
2022-09-14 13:35:46,151 INFO sqlalchemy.engine.Engine SELECT date, open FROM price WHERE ticker_id in (SELECT id FROM security WHERE ticker='A')
2022-09-14 13:35:46,152 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-09-14 13:35:46,221 INFO sqlalchemy.engine.Engine COMMIT


,date,open
0,2022-07-25,125.050003
1,2022-07-26,124.660004
2,2022-07-27,125.040001
3,2022-07-28,128.169998
4,2022-07-29,131.270004
5,2022-08-01,133.509995
6,2022-08-02,132.410004
7,2022-08-03,133.449997
8,2022-08-04,134.690002
9,2022-08-05,132.270004


In [38]:
with db.connect() as connection:
    with connection.begin():
        df_orders2      = pd.read_sql("SELECT date, low FROM price WHERE (date >= '2022-08-01' AND date <= '2022-08-22') AND (ticker_id in (SELECT id FROM security WHERE sector='Finance'))", connection)
df_orders2

2022-09-14 17:37:36,570 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-14 17:37:36,575 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-09-14 17:37:36,576 INFO sqlalchemy.engine.Engine [cached since 2.47e+04s ago] {'name': "SELECT date, low FROM price WHERE (date >= '2022-08-01' AND date <= '2022-08-22') AND (ticker_id in (SELECT id FROM security WHERE sector='Finance'))"}
2022-09-14 17:37:36,587 INFO sqlalchemy.engine.Engine SELECT date, low FROM price WHERE (date >= '2022-08-01' AND date <= '2022-08-22') AND (ticker_id in (SELECT id FROM security WHERE sector='Finance'))
2022-09-14 17:37:36,588 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-09-14 17:37:36,596 INFO sqlalchemy.engine.Engine COMMIT


,date,low
0,2022-08-01,9.860
1,2022-08-02,9.870
2,2022-08-03,9.870
3,2022-08-04,9.870
4,2022-08-05,9.870
5,2022-08-08,9.870
6,2022-08-09,9.875
7,2022-08-10,9.870
8,2022-08-11,9.870
9,2022-08-12,9.880
